In [1]:
import os
import sys
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from utils.files.file_helper import load_binary_file, save_binary_file
import utils.configuration
import pandas as pd
import concurrent.futures

In [3]:
config = utils.configuration.Configuration()

In [4]:
MERGED_DATA_LOCATION = config.config['DEFAULT']['MERGED_DATA_LOCATION']
MERGED_DATA_FILE_BIN = config.config['DEFAULT']['MERGED_DATA_FILE_BIN']
PREPROCESSED_DATA_FILE_BIN = config.config['DEFAULT']['PREPROCESSED_DATA_FILE_BIN']

In [5]:
def get_dependent_variable_value(cancelled, delay_duration):
    if cancelled == 1:
        return 'cancelled_flight'
    if delay_duration > 60:
        return 'long_delay'
    if delay_duration > 15:
        return 'delay'
    return 'no_delay'

In [6]:
def process_weather(data_df, prefix):
    data_df.loc[:, data_df.columns.str.startswith(prefix)] = data_df \
                                                .loc[:, data_df.columns.str.startswith(prefix)].fillna(value=0)
    
    data_df[prefix + 'fog'] = list(map(int , (data_df[prefix + 'wt01'] + data_df[prefix + 'wt02']).values > 0))
    data_df[prefix + 'hail'] = list(map(int , (data_df[prefix + 'wt04'] + data_df[prefix + 'wt05']).values > 0))
    data_df[prefix + 'damaging_wind'] = list(map(int , (data_df[prefix + 'wt10'] + data_df[prefix + 'wt11']).values > 0))
    data_df[prefix + 'snow'] = list(map(int , (data_df[prefix + 'snow'] + data_df[prefix + 'wt09']).values > 0))
    
    data_df = data_df.rename(columns={'snwd': 'snow_depth', 'awnd': 'average_wind_speed', 'wt03': 'thunder', 'wt07': 'dust', 'wt08': 'haze'})

    data_df[data_df.columns.drop(list(data_df.filter(regex=prefix + 'wt')))]
    
    return data_df.loc[:, ~data_df.columns.str.startswith(prefix + '_wt')]
    

In [7]:
def process_data_variables(file_data):
    
    data_df = file_data.copy()
    
    data_df.columns = map(str.lower, data_df.columns)
    
    logging.debug('Processing origin weather')
    
    data_df = process_weather(data_df, 'origin_')
    
    logging.debug('Processing dest weather')

    data_df = process_weather(data_df, 'dest_')
    
    logging.debug('Creating dependent variable')
    
    with concurrent.futures.ProcessPoolExecutor(16) as pool:
        data_df['status'] = list(pool.map(get_dependent_variable_value, data_df['cancelled'], data_df['arr_delay'], chunksize=1_000))
    
    logging.debug('Peforming final data preparation')
    
    data_df = data_df.drop(['dep_delay', 'arr_delay', 'cancelled', 'crs_elapsed_time', 
                           'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
                           'late_aircraft_delay'], axis=1)
    return data_df

In [8]:
def preprocess_data(merged_data_location, merged_data_file_name, preprocessed_file_name):
    logging.info('Loading historical data')

    parsed_data = load_binary_file(merged_data_location, merged_data_file_name)
    
    logging.info('Starting to process data file')
    
    processed_data = process_data_variables(parsed_data)
    
    logging.info('Saving preprocessed data file')
    
    save_binary_file(processed_data, preprocessed_file_name, merged_data_location)

In [ ]:
preprocess_data(MERGED_DATA_LOCATION, MERGED_DATA_FILE_BIN, PREPROCESSED_DATA_FILE_BIN)

INFO:root:Loading historical data
INFO:root:Starting to process data file
